<a href="https://colab.research.google.com/github/VaidehiBhat/Sample101vedansh/blob/origin%2Fmain/PreProcessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

def preprocessor(signal_input):
  import numpy as np
  signal_input=np.array(signal_input)
  #Mean Correction:
  signal_input= signal_input-np.mean(signal_input)
  #Notch Filter:
  import scipy.signal as signal
  fs =  2000
  f0 = 50.0/(fs/2)
  Q = 30.0

  b1, a1 = signal.iirnotch(f0, Q)

  freq, h = signal.freqz(b1, a1, fs=fs)
  signal_input = signal.filtfilt(b1, a1, signal_input)
  #Butterworth Low Pass Filter:
  cutoff_freq=400
  sampling_rate=2000
  nyquist = 0.5 * sampling_rate
  normal_cutoff = cutoff_freq / nyquist
  b, a = signal.butter(1, normal_cutoff, btype='low', analog=False)
  signal_input = signal.filtfilt(b, a, signal_input)
  #PyWavelets:
  import pywt
  import math

  wavelet_type='sym8'
  sub_coeff_of_decomp=2
  decision_type='soft'
  wavelet_type = wavelet_type
  w = pywt.Wavelet(wavelet_type)
  maxlev = pywt.dwt_max_level(len(signal_input), w.dec_len) - sub_coeff_of_decomp
  coeffs = pywt.wavedec(signal_input, wavelet_type, level=maxlev)

  for i in range(1, len(coeffs)):
      M = len(coeffs[i])
      lambda_val = math.sqrt(2*math.log(M))
      coeffs[i] = pywt.threshold(coeffs[i], lambda_val, decision_type)
      signal_input = pywt.waverec(coeffs, wavelet_type)
  #Baseline Removal:
  noise_arr=[signal_input[1000:1200] , signal_input[2000:2200] , signal_input[3000:3200] , signal_input[4000:4200]]
  mean_noise_arr=[]
  for i in range(200):
    mean_noise_arr.append((noise_arr[0][i]+noise_arr[1][i]+ noise_arr[2][i] + noise_arr[3][i]) / 4)
  mean_noise_arr=np.array(mean_noise_arr)
  populated_mean_noise=[]
  for i in range(80):
    for j in range(200):
      populated_mean_noise.append(mean_noise_arr[j])
  signal_input=signal_input-populated_mean_noise

  return signal_input



